In [ ]:
from OptimizedDNN import *
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('../data/EURUSD_HOUR.csv', parse_dates=['time'], index_col='time')
dataset = data.rename(columns={'price': 'Price'})
print(dataset)

In [ ]:
df = dataset.copy()
WINDOW = 50
FAST_SMA = 75
SLOW_SMA = 150
FAST_EMA = 12
SLOW_EMA = 26
SIGNAL_EMA = 9
RSI_WINDOW = 14

# Log Returns
df['Returns'] = np.log(df['Price'] / df['Price'].shift(1))
# Direction for class weight balancing to eliminate buy bias
df['Direction'] = np.where(df['Returns'] > 0, 'buy', 'sell')
# MACD Growth Indicator
macd = df['Price'].ewm(span=FAST_EMA, adjust=False).mean() - df['Price'].ewm(span=SLOW_EMA, adjust=False).mean()
signal = macd.ewm(span=SIGNAL_EMA, adjust=False).mean()
df['MACD'] = macd - signal
# SMA Crossover with 75-150 Fast SMA-Slow SMA split
df['SMA Crossover'] = df['Price'].rolling(FAST_SMA).mean() - df['Price'].rolling(SLOW_SMA).mean()
# Mean Reversion (similar to Bollinger Bands) with 50-period window
df['Mean Reversion'] = (df['Price'] - df['Price'].rolling(WINDOW).mean()) / df['Price'].rolling(WINDOW).std()
# Rolling Min/Max normalization with current price
df['Rolling Min'] = (df['Price'].rolling(WINDOW).min() / df['Price']) - 1
df['Rolling Max'] = (df['Price'].rolling(WINDOW).max() / df['Price']) - 1
# Momentum
change = df['Price'].diff()
df['RSI'] = 100 - (100 / (1 + (change.mask(change < 0, 0.0).rolling(RSI_WINDOW).mean() / -change.mask(change > 0, -0.0).rolling(RSI_WINDOW).mean())))
# Volatility
df['Volatility'] = df['Returns'].rolling(WINDOW).std()

In [ ]:
encoder = LabelEncoder()
df['Direction'] = encoder.fit_transform(df['Direction'])

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
lags = 8
columns = []
features = ['Returns', 'Direction', 'MACD', 'SMA Crossover', 'Mean Reversion', 'Rolling Min', 'Rolling Max', 'RSI', 'Volatility']

for feature in features:
    for lag in range(1, lags + 1):
        column = '{}_lag_{}'.format(feature, lag)
        df[column] = df[feature].shift(lag)
        columns.append(column)
df.dropna(inplace=True)
df

In [ ]:
split = int(len(df) * 0.8)
training_set = df.iloc[:split].copy()
test_set = df.iloc[split:].copy()

mu = training_set.mean()
sigma = training_set.std()
standardized_training_set = (training_set - mu) / sigma

test_mu = test_set.mean()
test_sigma = test_set.std()
standardized_test_set = (test_set - test_mu) / test_sigma

X_train = standardized_training_set[columns]
X_test = standardized_test_set[columns]
y_train = training_set['Direction']
y_test = test_set['Direction']

y_train

In [ ]:
param_grid = {
    'hl': [1, 2, 3, 4, 5],
    'hu': [25, 50, 100, 150],
    'dropout': [True, False],
    'rate': [0.1, 0.2, 0.3, 0.4, 0.5],
    'regularize': [True, False],
    'reg': [l1(0.0001), l2(0.0001)],
}

In [ ]:
grid_search = GridSearchCV(
    estimator=create_keras_model(input_dim=X_train.shape[1]),
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=3),  # Use StratifiedKFold since it's a classification task
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_

In [ ]:
accuracy = best_model.score(X_test, y_test)
print("Best Parameters:", best_params)